# 15 puzzle

<img src="img/1.jpg">

The 15 puzzle, a classic tile-sliding challenge, boasts countless variations of the "solved" state. I'm focusing on the iconic arrangement pictured above, where tiles 1 through 15 slide across a 4x4 grid, leaving one space vacant.

The rules are straightforward: move any tile adjacent to the empty space into it. In the solved state, for example, you can swap the 15 rightward or the 12 downward.

No matter how scrambled the tiles, this program will chart the shortest path back to order.

## Algorithm
In addition to the representation work that I'm about to do, the main algorithm for solving the puzzle is A* search. The reason for this is that the 15 puzzle is basically a search problem. The heuristic and all the details will be explained on each step of this notebook.